In [9]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [10]:
from langchain.chat_models import ChatOpenAI

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
KEY = os.getenv("TOGETHER_API_KEY")

In [13]:
print(KEY)

7ebf4022abdc280634e753f9db99f15a862a70e78e02ca93ae22195aa3503803


In [14]:
llm = ChatOpenAI(
    openai_api_key=KEY, 
    model_name="mistralai/Mistral-7B-Instruct-v0.2",  # or any Together model
    openai_api_base="https://api.together.xyz/v1",    # <-- required
    temperature=0.3
)


In [15]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1431cfc10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x141b07bb0>, model_name='mistralai/Mistral-7B-Instruct-v0.2', temperature=0.3, openai_api_key='7ebf4022abdc280634e753f9db99f15a862a70e78e02ca93ae22195aa3503803', openai_api_base='https://api.together.xyz/v1', openai_proxy='')

In [16]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [17]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [18]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [33]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=False)

In [34]:
quiz_chain

LLMChain(prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1431cfc10>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x141b07bb0>, model_name='mistralai/Mistral-7B-Instruct-v0.2', temperature=0.3, openai_api_key='7ebf4022abdc280634e753f9db99f15a862a70e78e02ca93ae22195aa3503803', openai_api_base='https://api.together.xyz/v1', openai_proxy=''), output_key='quiz')

In [35]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [36]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [37]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=False)

In [38]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=False,)

In [39]:
file_path = r"/Users/swatimittal/mcqgen/data.txt"

In [40]:
file_path

'/Users/swatimittal/mcqgen/data.txt'

In [41]:
with open(file_path, 'r') as file:
    text = file.read()

In [42]:
print(text)

Machine Learning is mainly divided into three core types: Supervised, Unsupervised and Reinforcement Learning along with two additional types, Semi-Supervised and Self-Supervised Learning.

Supervised Learning: Trains models on labeled data to predict or classify new, unseen data.
Unsupervised Learning: Finds patterns or groups in unlabeled data, like clustering or dimensionality reduction.
Reinforcement Learning: Learns through trial and error to maximize rewards, ideal for decision-making tasks.
Note: The following are not part of the original three core types of ML, but they have become increasingly important in real-world applications, especially in deep learning.

Additional Types:

Self-Supervised Learning: Self-supervised learning is often considered a subset of unsupervised learning, but it has grown into its own field due to its success in training large-scale models. It generates its own labels from the data, without any manual labeling. 
Semi-Supervised Learning: This approa

In [43]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [44]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [47]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": text,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )
    print("Quiz:\n", response["quiz"])
    print("\nReview:\n", response["review"])
    print("\nToken usage:", cb)


Quiz:
  ### RESPONSE_JSON
{
  "1": {
    "mcq": "Which of the following is a core type of Machine Learning that trains models on labeled data to predict or classify new, unseen data?",
    "options": {"a": "Supervised Learning", "b": "Unsupervised Learning", "c": "Reinforcement Learning", "d": "Self-Supervised Learning"},
    "correct": "a"
  },
  "2": {
    "mcq": "Which of the following Machine Learning approach combines a small amount of labeled data with a large amount of unlabeled data?",
    "options": {"a": "Supervised Learning", "b": "Unsupervised Learning", "c": "Reinforcement Learning", "d": "Semi-Supervised Learning"},
    "correct": "d"
  },
  "3": {
    "mcq": "In which type of Machine Learning does the model learn through trial and error to maximize rewards, ideal for decision-making tasks?",
    "options": {"a": "Supervised Learning", "b": "Unsupervised Learning", "c": "Reinforcement Learning", "d": "Self-Supervised Learning"},
    "correct": "c"
  },
  "4": {
    "mcq":

In [48]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2053
Prompt Tokens:1124
Completion Tokens:929
Total Cost:0.0


In [65]:
response

{'text': 'Machine Learning is mainly divided into three core types: Supervised, Unsupervised and Reinforcement Learning along with two additional types, Semi-Supervised and Self-Supervised Learning.\n\nSupervised Learning: Trains models on labeled data to predict or classify new, unseen data.\nUnsupervised Learning: Finds patterns or groups in unlabeled data, like clustering or dimensionality reduction.\nReinforcement Learning: Learns through trial and error to maximize rewards, ideal for decision-making tasks.\nNote: The following are not part of the original three core types of ML, but they have become increasingly important in real-world applications, especially in deep learning.\n\nAdditional Types:\n\nSelf-Supervised Learning: Self-supervised learning is often considered a subset of unsupervised learning, but it has grown into its own field due to its success in training large-scale models. It generates its own labels from the data, without any manual labeling. \nSemi-Supervised L

In [68]:
quiz = response.get("quiz")

In [69]:
import json

# Clean and parse
clean_quiz = quiz.replace("### RESPONSE_JSON", "").strip()
quiz_json = json.loads(clean_quiz)

# Pretty print
print(json.dumps(quiz_json, indent=2))


{
  "1": {
    "mcq": "Which of the following is a core type of Machine Learning that trains models on labeled data to predict or classify new, unseen data?",
    "options": {
      "a": "Supervised Learning",
      "b": "Unsupervised Learning",
      "c": "Reinforcement Learning",
      "d": "Self-Supervised Learning"
    },
    "correct": "a"
  },
  "2": {
    "mcq": "Which of the following Machine Learning approach combines a small amount of labeled data with a large amount of unlabeled data?",
    "options": {
      "a": "Supervised Learning",
      "b": "Unsupervised Learning",
      "c": "Reinforcement Learning",
      "d": "Semi-Supervised Learning"
    },
    "correct": "d"
  },
  "3": {
    "mcq": "In which type of Machine Learning does the model learn through trial and error to maximize rewards, ideal for decision-making tasks?",
    "options": {
      "a": "Supervised Learning",
      "b": "Unsupervised Learning",
      "c": "Reinforcement Learning",
      "d": "Self-Supervi

In [72]:
quiz_table_data = []
for key, value in quiz_json.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [73]:
quiz_table_data

[{'MCQ': 'Which of the following is a core type of Machine Learning that trains models on labeled data to predict or classify new, unseen data?',
  'Choices': 'a: Supervised Learning | b: Unsupervised Learning | c: Reinforcement Learning | d: Self-Supervised Learning',
  'Correct': 'a'},
 {'MCQ': 'Which of the following Machine Learning approach combines a small amount of labeled data with a large amount of unlabeled data?',
  'Choices': 'a: Supervised Learning | b: Unsupervised Learning | c: Reinforcement Learning | d: Semi-Supervised Learning',
  'Correct': 'd'},
 {'MCQ': 'In which type of Machine Learning does the model learn through trial and error to maximize rewards, ideal for decision-making tasks?',
  'Choices': 'a: Supervised Learning | b: Unsupervised Learning | c: Reinforcement Learning | d: Self-Supervised Learning',
  'Correct': 'c'},
 {'MCQ': 'Which of the following Machine Learning techniques generates its own labels from the data, without any manual labeling?',
  'Choic

In [74]:
df=pd.DataFrame(quiz_table_data)

In [75]:
df

,MCQ,Choices,Correct
0,Which of the following is a core type of Machi...,a: Supervised Learning | b: Unsupervised Learn...,a
1,Which of the following Machine Learning approa...,a: Supervised Learning | b: Unsupervised Learn...,d
2,In which type of Machine Learning does the mod...,a: Supervised Learning | b: Unsupervised Learn...,c
3,Which of the following Machine Learning techni...,a: Supervised Learning | b: Unsupervised Learn...,d
4,Which of the following is a subset of unsuperv...,a: Supervised Learning | b: Unsupervised Learn...,d


In [76]:
df.to_csv("Data_Science_Quiz.csv",index=False)